In [10]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis --unzip

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0




  0%|          | 0.00/1.99M [00:00<?, ?B/s]
100%|██████████| 1.99M/1.99M [00:00<00:00, 10.7MB/s]
100%|██████████| 1.99M/1.99M [00:00<00:00, 10.6MB/s]


In [21]:
import pandas as pd
training_data = pd.read_csv("twitter_training.csv")

In [22]:
# what is the data liek ?
training_data
# splitting into valadation data
validation_data=pd.read_csv("twitter_validation.csv")
validation_data

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
994,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
997,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [23]:

#lets chnage the colum names
training_data.columns=['SerialNo','Company','labels','text']
validation_data.columns=['SerialNo','Company','labels','text']

In [24]:
#lets see how many null vals are there
print(training_data.isnull().sum())
# now the same for valadation data
validation_data.isnull().sum()

SerialNo      0
Company       0
labels        0
text        686
dtype: int64


SerialNo    0
Company     0
labels      0
text        0
dtype: int64

In [25]:
training_data.duplicated().sum()
validation_data.duplicated().sum()
sentiment=pd.concat([training_data,validation_data], ignore_index=True)

In [26]:
sentiment.head(10)

,SerialNo,Company,labels,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
5,2402,Borderlands,Positive,So I spent a few hours making something for fu...
6,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
7,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
8,2402,Borderlands,Positive,So I spent a few hours making something for fu...
9,2402,Borderlands,Positive,2010 So I spent a few hours making something f...


In [27]:
sentiment.dropna(inplace=True)
sentiment.drop_duplicates(inplace=True)
sentiment.shape

(72138, 4)

In [28]:
# now lests look at the data once more ....
print(sentiment.head(10))
# now lets drop siralnumber adn company
sentiment.drop(['SerialNo','Company'],axis=1,inplace=True)

   SerialNo      Company    labels  \
0      2401  Borderlands  Positive   
1      2401  Borderlands  Positive   
2      2401  Borderlands  Positive   
3      2401  Borderlands  Positive   
4      2401  Borderlands  Positive   
5      2402  Borderlands  Positive   
6      2402  Borderlands  Positive   
7      2402  Borderlands  Positive   
8      2402  Borderlands  Positive   
9      2402  Borderlands  Positive   

                                                text  
0  I am coming to the borders and I will kill you...  
1  im getting on borderlands and i will kill you ...  
2  im coming on borderlands and i will murder you...  
3  im getting on borderlands 2 and i will murder ...  
4  im getting into borderlands and i can murder y...  
5  So I spent a few hours making something for fu...  
6  So I spent a couple of hours doing something f...  
7  So I spent a few hours doing something for fun...  
8  So I spent a few hours making something for fu...  
9  2010 So I spent a few hours 